# Static Lineout Notebook -> Largely the same but modified to work with two azavs from the same image

## Import python libraries

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import xrayscatteringtools as xrst
import warnings
xrst.enable_underscore_cleanup()

## Next define keys (data parameters) to be loaded and define function to load data
- keys_to_combine: some keys loaded for each shot & stored per shot
- keys_to_sum: some keys loaded per each run and added
- keys_to_check : check if some keys exist and have same values in all runs and load these keys 

# Load the data into defined variables
Sometimes keys of a data parameter may be saved with different name, print out all keys to figure out the required key

In [ ]:
###############################################
runNumbers = []  # Enter a single run number here
folders = xrst.get_data_paths(runNumber) # Defaults to the info in config.yaml. You can overwrite this with strings, character arrays, or lists of either.
runtype = xrst.get_config_for_runs(runNumber,'samples','sample') # Edit this for the type of run, or auto load from config.yaml
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exist and have same values in all runs and load these keys 
_keys_to_combine = [
    'jungfrau4M/azav_mask0_azav',
    'jungfrau4M/azav_mask1_azav',
    'lightStatus/laser',
    'lightStatus/xray'
]

_keys_to_sum = [
    'Sums/jungfrau4M_calib', # No lower ADU threshold
    'Sums/jungfrau4M_calib_xrayOn_thresADU1' # X-rays on, ADU lower threshold
]

_keys_to_check = [
    'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_q',
    'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask',
    'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask',
    'UserDataCfg/jungfrau4M/cmask',
]
# Load the data in
_data = xrst.combineRuns(runNumbers, folders, _keys_to_combine, _keys_to_sum, _keys_to_check, verbose=False)  # this is the function to load the data with defined keys

# String for nice things
runNumbersRange = xrst.compress_ranges(runNumbers)
runType = xrst.get_config_for_runs(runNumbers[0],'samples','sample') # Default to information in the first run you load.
niceTitle = f"{xrst.get_config('expNumber')} : {'Run' if np.size(runNumbers)==1 else 'Runs'} {runNumbersRange} : {runType}"

# Unfiltered data
azav_unfiltered = np.squeeze(_data['jungfrau4M/azav_mask0_azav']) # I(q) : 1D azimuthal average of signals in each q bin
unfiltered_mask = _data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask'].astype(bool)
# Filtered data
azav_filtered = np.squeeze(_data['jungfrau4M/azav_mask1_azav']) # I(q) : 1D azimuthal average of signals in each q bin
filtered_mask = _data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask'].astype(bool)

q = _data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_q'] # q bin centers

laserOn = _data['lightStatus/laser'].astype(bool)  # laser on events 
xrayOn = _data['lightStatus/xray'].astype(bool)  # xray on events
jungfrau_sum = _data['Sums/jungfrau4M_calib_xrayOn_thresADU1']  # Total Jungfrau detector counts summed in a run
cmask = _data['UserDataCfg/jungfrau4M/cmask'].astype(bool) # Combined mask

# Plot the Masked Jungfrau4M Sums as a check, if you want

In [ ]:
###############################################
if True: # For plotting or not
###############################################
    _masked_sum_unfiltered = np.copy(jungfrau_sum)
    _masked_sum_filtered = np.copy(jungfrau_sum)
    _masked_sum_unfiltered[~(cmask & unfiltered_mask).astype(bool)] = np.nan
    _masked_sum_filtered[~(cmask & filtered_mask).astype(bool)] = np.nan
    plt.figure(figsize=(20,8))
    plt.subplot(1,2,1)
    pcm = xrst.plot_j4m(_masked_sum_unfiltered,vmin=0,vmax=np.nanpercentile(_masked_sum_unfiltered,99.5))
    plt.colorbar(pcm)
    plt.title('Unfiltered Jungfrau4M Sum')

    plt.subplot(1,2,2)
    pcm = xrst.plot_j4m(_masked_sum_filtered,vmin=0,vmax=np.nanpercentile(_masked_sum_filtered,99.5))
    plt.colorbar(pcm)
    plt.title('Filtered Jungfrau4M Sum')

    plt.suptitle(niceTitle)
    plt.show()

# Plot Single Run average azavs

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    azav_avg_unfiltered = np.nanmean(azav_unfiltered[xrayOn],axis=0) # Averaging the shot to shot azav data, no other normalization done here. This is purely to measure average signal per shot.
    azav_avg_filtered = np.nanmean(azav_filtered[xrayOn],axis=0)
    plt.figure(figsize=[12,5])
    plt.subplot(1,2,1)
    plt.plot(q, np.log10(azav_avg_unfiltered)) # Plotting it
    plt.xlabel('q')
    plt.ylabel('log[I]')
    plt.title('Unfiltered')
    
    plt.subplot(1,2,2)
    plt.plot(q, np.log10(azav_avg_filtered)) # Plotting it
    plt.xlabel('q')
    plt.ylabel('log[I]')
    plt.title('Filtered')
    
    plt.suptitle(niceTitle)
    plt.show()

## Save the azavs data to a folder

In [ ]:
##########################
runFolderUnfiltered = 'Azavs/Unfiltered' # Edit this if you want
runFolderFiltered = 'Azavs/Filtered' # Edit this if you want
##########################
os.makedirs(runFolderUnfiltered, exist_ok=True)
os.makedirs(runFolderFiltered, exist_ok=True)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    np.savez(f'{runFolderUnfiltered}/Run_{runNumbers[0]}_{runtype}.npz',x=q,y=np.log10(azav_avg_unfiltered))
    np.savez(f'{runFolderFiltered}/Run_{runNumbers[0]}_{runtype}.npz',x=q,y=np.log10(azav_avg_filtered))

## Plot data for each run in the folder

In [ ]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import xrayscatteringtools as xrst

runFolderUnfiltered = 'Azavs/Unfiltered'  # Edit this if you want
runFolderFiltered = 'Azavs/Filtered'

plt.figure(figsize=[12,5])
plt.subplot(1,2,1)
# Regular expression pattern for matching filenames
pattern = re.compile(r'^Run_(\d+)_(.+?)\.npz$')

# --- BUILD A SORTABLE LIST OF (run_number, filename) ---
files = []
for filename in os.listdir(runFolderUnfiltered):
    match = pattern.match(filename)
    if match:
        run_number = int(match.group(1))
        files.append((run_number, filename))
    else:
        print(f"Skipped File: {filename}")

# --- SORT BY RUN NUMBER ---
files.sort(key=lambda x: x[0])

# Loop through sorted files
for run_number, filename in files:
    title = filename.replace(f"Run_{run_number}_", "").replace(".npz", "")
    file_path = os.path.join(runFolderUnfiltered, filename)
    data = np.load(file_path)

    plt.plot(data['x'], data['y'],
             linestyle='-', marker='o', linewidth=1.5,
             label=f'Run {run_number} {title}')

# Make plot look nice
plt.xlabel(r'q $\AA^{-1}$')
plt.ylabel('log[I]')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)) # The legends should be the same on both plots so we only need the second one
plt.title('Unfiltered')

plt.subplot(1,2,2)
# Regular expression pattern for matching filenames
pattern = re.compile(r'^Run_(\d+)_(.+?)\.npz$')

# --- BUILD A SORTABLE LIST OF (run_number, filename) ---
files = []
for filename in os.listdir(runFolderFiltered):
    match = pattern.match(filename)
    if match:
        run_number = int(match.group(1))
        files.append((run_number, filename))
    else:
        print(f"Skipped File: {filename}")

# --- SORT BY RUN NUMBER ---
files.sort(key=lambda x: x[0])

# Loop through sorted files
for run_number, filename in files:
    title = filename.replace(f"Run_{run_number}_", "").replace(".npz", "")
    file_path = os.path.join(runFolderFiltered, filename)
    data = np.load(file_path)

    plt.plot(data['x'], data['y'],
             linestyle='-', marker='o', linewidth=1.5,
             label=f'Run {run_number} {title}')

# Make plot look nice
plt.xlabel(r'q $\AA^{-1}$')
plt.ylabel('log[I]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Filtered')
plt.suptitle(f"{xrst.get_config('expNumber')} Runs ")
plt.show()